<a href="https://colab.research.google.com/github/akaravi/NTK.Education.Practice/blob/main/InformationSearchAndRetrieval/2-14040128/Word2Vec_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implement Word2Vec

In this assignment, you are required to implement the `Word2Vec` class from scratch using Python.

Please complete the methods inside the `Word2Vec` class:
- `__init__`: Initialize weights
- `softmax`: Impliment softmax activation function
- `train`: Loop over the data and optimize your model

You may use NumPy, but **do not** use any external word embedding libraries like Gensim or Torch for this task.

Good luck!


# Skip Gram Implementation

In [21]:
import re
import numpy as np
from collections import defaultdict
import random
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [22]:
class Vocabulary:
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.word_count = defaultdict(int)
        self.total_words = 0
        self.vocab_size = 0

    def build_vocab(self, sentences, min_count=2):
        # Count words
        for word in sentences:
            self.word_count[word] += 1

        # Create word2idx and idx2word mapping
        idx = 0
        for word, count in self.word_count.items():
            if count >= min_count:
              self.word2idx.update({word: idx})
              idx += 1

        # self.word2idx = {word: idx for idx, (word, count) in enumerate(self.word_count.items()) if count >= min_count}
        self.idx2word = {idx: word for word, idx in self.word2idx.items()}
        self.vocab_size = len(self.word2idx)
        self.total_words = sum([count for word, count in self.word_count.items() if count >= min_count])

    def word_to_index(self, word):
        return self.word2idx.get(word, -1)

    def index_to_word(self, index):
        return self.idx2word.get(index, None)


In [23]:
def generate_training_data(vocab, sentences, window_size=2):
    training_data = []
    sentence_indices = [vocab.word_to_index(word) for word in sentences if vocab.word_to_index(word) != -1]

    for center_idx, center_word in enumerate(sentence_indices):
        context_start = max(0, center_idx - window_size)
        context_end = min(len(sentence_indices), center_idx + window_size + 1)

        for context_idx in range(context_start, context_end):
            if context_idx != center_idx:
                context_word = sentence_indices[context_idx]
                training_data.append((center_word, context_word))

    return np.array(training_data)


شروع تمرین
######################################
######################################
######################################

In [24]:
class Word2Vec:
    def __init__(self, vocab_size, embed_size=100, learning_rate=0.001):
        #Start
        # سایز زا مشخص می کنم
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.learning_rate = learning_rate

        # مقداردهی اولیه ماتریس‌های وزن
        #ورودی مخفی
        self.W = np.random.uniform(-0.5, 0.5, (vocab_size, embed_size))
        #خروجی
        self.W_prime = np.random.uniform(-0.5, 0.5, (embed_size, vocab_size))
        #End




    def softmax(self, x):
        #Start
        xMax=np.max(x)
        retOut = np.exp(x - xMax)
        return retOut / np.sum(retOut)
        #ENd



    def train(self, training_data, epochs=1000):
        #Start
        #تعداد تکرار برای بررسی نتججه
        for epochNumber in range(epochs):
            myloss = 0
            #بررسی وزن ها
            for c_word, context_word in training_data:
                hValue = self.W[c_word]
                dispacher = np.dot(hValue, self.W_prime)
                y_pred = self.softmax(dispacher)

                #encoding
                y_true = np.zeros(self.vocab_size)
                y_true[context_word] = 1

                # مجاسبه خطا
                er = y_pred - y_true

                # برروز رسانی وزن ها
                self.W_prime -= self.learning_rate * np.outer(hValue, er)
                self.W[c_word] -= self.learning_rate * np.dot(self.W_prime, er)

                # محاسبه looss
                myloss -= np.log(y_pred[context_word])

            if epochNumber % 100 == 0:
                print(f'Epoch Count {epochNumber}, Calculat Loss: {myloss}')




######################################
######################################
######################################
پایان تمرین

In [25]:
# Reading the Persian stopwords from the file
from google.colab import files
# بارگذاری فایل
uploaded = files.upload()
# نام فایل بارگذاری شده
filename = list(uploaded.keys())[0]

with open(filename, 'r', encoding='utf-8') as f:
    stopwords = set(f.read().splitlines())  # Using a set for faster lookup

# Reading the Persian stopwords from the file
#with open('persian.txt', 'r', encoding='utf-8') as f:
#    stopwords = set(f.read().splitlines())  # Using a set for faster lookup

# Sample Persian text
text = "ملکه و زن ها در کنار همسران و خانواده خود یعنی شاه و مرد ها در یک سرزمین پهناور زندگی می‌کردند شاه همیشه به مرد ها تذکر میداد که قدرت در اتحاد مرد ها و شاه نهفته است و در این قلمرو ملکه به زن ها یادآوری می‌کرد که همبستگی زن ها و ملکه مهم است و در این داستان هر مرد که نزد شاه یا زن که نزد ملکه می‌آمد از آنها حکم می‌گرفت تا به دیگران کمک کنند شاه عادل و قادر بود و ملکه خردمند و زیبا و هر مرد که از حکمت شاه یا زن که از عدالت ملکه راضی نبود نزد آنها می‌رفت تا شکایت خود را مطرح کند شاه و مرد و ملکه و زن در کنار هم بودند و هیچ کس از شاه یا ملکه نمی‌ترسید شاه همیشه به زبان میاورد که مرد ها باید به یکدیگر کمک کنند و ملکه تأکید داشتند زن ها هم باید متحد باشند"

# Tokenizing the text (you can modify the tokenizer if needed)
words = text.split()

# Removing stopwords
filtered_text = [word for word in words if word not in stopwords]

# Joining the words back into a sentence
cleaned_text = ' '.join(filtered_text)

print(cleaned_text)


Saving persian.txt to persian (3).txt
ملکه زن همسران خانواده شاه مرد سرزمین پهناور زندگی می‌کردند شاه مرد تذکر میداد قدرت اتحاد مرد شاه نهفته قلمرو ملکه زن یادآوری می‌کرد همبستگی زن ملکه مهم داستان مرد شاه زن ملکه می‌آمد حکم می‌گرفت کمک شاه عادل قادر ملکه خردمند زیبا مرد حکمت شاه زن عدالت ملکه راضی می‌رفت شکایت مطرح شاه مرد ملکه زن شاه ملکه نمی‌ترسید شاه زبان میاورد مرد کمک ملکه تأکید زن متحد


In [26]:
# wiki_dump_path = 'enwiki-latest-pages-articles.xml.bz2'  # Path to your Wikipedia dump file

# # Load and preprocess the dataset
# sentences = list(load_wiki_data(wiki_dump_path))

# Build the vocabulary
vocab = Vocabulary()
vocab.build_vocab(cleaned_text.split(' '))

# Generate training data
training_data = generate_training_data(vocab, cleaned_text.split(' '))

# Initialize and train Word2Vec model
word2vec_model = Word2Vec(vocab.vocab_size)
word2vec_model.train(training_data, epochs=1000)


Epoch Count 0, Calculat Loss: 230.77703564149627
Epoch Count 100, Calculat Loss: 181.62920376715596
Epoch Count 200, Calculat Loss: 181.15030113476672
Epoch Count 300, Calculat Loss: 180.99222678449007
Epoch Count 400, Calculat Loss: 180.91428562127163
Epoch Count 500, Calculat Loss: 180.86834024484958
Epoch Count 600, Calculat Loss: 180.83828187711083
Epoch Count 700, Calculat Loss: 180.81722031415802
Epoch Count 800, Calculat Loss: 180.80172673746844
Epoch Count 900, Calculat Loss: 180.78990764937774


Defines a function to retrieve the word embedding for a given word from a Word2Vec model. If the word exists in the vocabulary, its corresponding vector is returned; otherwise, None is returned.

In [27]:
def get_word_embedding(word, vocab, model):
    word_idx = vocab.word_to_index(word)
    if word_idx != -1:
        return model.W[word_idx]
    else:
        return None

embedding = get_word_embedding("مرد", vocab, word2vec_model)
print(embedding)


[ 0.159925   -0.15450832 -0.28439054 -0.1401933  -0.27496334  0.40087362
  0.03902469 -0.03101691  0.12021181 -0.20226868 -0.35447277  0.18648668
 -0.36644978 -0.47050775 -0.22197275  0.3211393   0.19444794  0.10514901
  0.33220747  0.13687489 -0.46719954  0.15604795 -0.46981665 -0.0716535
  0.40819045 -0.06128793 -0.43219489 -0.04791149  0.50205303 -0.00613901
 -0.352595   -0.23070617  0.13942727 -0.4925073  -0.48999409 -0.24486596
  0.24898835  0.15138764 -0.21567084 -0.27638513 -0.41530056 -0.08327305
  0.33147747 -0.40194631  0.42548831 -0.23805968  0.25196989 -0.06786954
  0.32279015 -0.48934354  0.01772535 -0.33292514 -0.38998284 -0.40271795
 -0.46135411  0.47493908  0.37104209 -0.00138381  0.11060302  0.52683122
 -0.14220342  0.40896466  0.29971609 -0.42319487  0.30725927 -0.12376862
  0.48694662  0.12383187  0.0829087  -0.15987773  0.47145945 -0.40901381
  0.41801148  0.20151515  0.1297566   0.29990892 -0.13765469  0.24024289
 -0.0258212   0.22264424 -0.02229593 -0.27951941  0.